## Data Modeling using Cassandra DataBase

In this project, you'll apply what you've learned on data modeling with Apache Cassandra and complete an ETL pipeline using Python. 

- model your data by creating tables in Apache Cassandra to run queries. 
- You are provided with part of the ETL pipeline that transfers data from a set of CSV files within a directory to        create a streamlined CSV file to model and insert data into Apache Cassandra tables. 

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collecting each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    
print(file_path_list[0], len(file_path_list))

/home/workspace
/home/workspace/event_data/2018-11-27-events.csv 30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if needed to get total number of rows 
#print("Full data rows = ", len(full_data_rows_list))

# uncomment the code below if needed view the rows 
#print("Full data rows = ", full_data_rows))


# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables

## The filter will be: select only records that 'artist' has value then result at smaller one

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))       


In [4]:
# check the number of rows csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Modelling Apache Cassandra 
 As Apache Cassandra is a a NoSQL database, think about the query first which will be used to fetch the data based on  which we will create the Table required.

 The main characteristc of Apache Cassandra,  to efficiently retrieve data, the where clause in fetch query must contain all the composite partition keys in the same order as specified in the primary key definition. If this is not provided, will result in error.
 
Working with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Understanding the data

In [5]:
## understangin data using Pandas
event = pd.read_csv('event_datafile_new.csv')  
event.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


#### Creating a Cluster

In [10]:
# Make a connection to a Cassandra instance 

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [11]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [12]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Create tables to run the following queries.  



### Query 1
#### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4. 

> a. The expected output is : "Name of the artist, title of the song and length of the track"    
> b. Based on : "sessionId and itemInSession"

From the above two points the query to get the data will be a SELECT statement like :

>SELECT Name of the artist, title of the song, length of the track FROM TABLE_NAME  
>       WHERE sessionId = value AND itemInSession = value

##### Points of interest

<ul>
<li>As the filter will be done based on session_id, the table name for this query will be "Session_library"</li>
<li>The table will be created only if it does not exist.<\li>
<li>The PRIMARY KEY will be <em><strong>(sessionId and itemInSession)</strong></em> that are enougth to get unique values for this table.</li>
<li>The query result need only artist, song title and song's length, then they are the table columns.</li>
<\ul>


### Query 2
#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
>a. The expected output is : "artist's name, song's title and user: first and last name"    
>b. Based on : "userid and sessionid" - sorted by itemInSession

From the above two points the query to get the data will be a SELECT statement like :

>Select artist name, song's title, user's firstName, user's lastName from Table_name WHERE userId = value and sessionId = value"

##### Points of interest

<ul>
<li>As the filter will be done based on userId, the table name for this query will be "User_library"</li>
<li>The table will be created only if it does not exist.<\li>
<li>The PRIMARY KEY will be <em><strong>((userId, sessionId), itemInSession))</strong></em> that are enougth to get unique values for this table.<\li>
<li>Note that the cluster will be  <em><strong>userId and sessionID</strong></em>  and the column <em><strong>itemInSession</em></strong> will be used for sorted ascending.<\li>
<li>The query result need only artist's name, song's title and user's name (first and last), then they are the table columns.</li>
<\ul>


### Query 3
#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

>a. The expected output is : "user: first and last name"    
>b. Based on : "song's title" 
    
From the above two points the query to get the data will be a SELECT statement like :

>Select firstName, lastName from song_library WHERE song = value"

##### Points of interest

<ul>
<li>As the filter will be done based on song's title, the table name for this query will be "Song_library"</li>
<li>The table will be created only if it does not exist.<\li>
<li>The PRIMARY KEY will be <em><strong>(song, userId)</strong></em> that are enougth to get unique values for this table.<\li>
<li>The query result need only user's name (first and last), then they are the table columns.</li>
<li>Note that, regardless how many times the user listened the song, the table will record only once.
<\ul>

### Create tables

In [13]:
queries = []  # list of query create tables

## Query 1:  Give me the artist, song title and song's length in the music app history 
##                  that was heard during sessionId = 338, and itemInSession = 4


query1 = "CREATE TABLE IF NOT EXISTS session_library "
query1 = query1 + """(sessionId int, itemInSession int, artist text, song text, length float, 
                PRIMARY KEY (sessionId, itemInSession))"""

queries.append(query1)

## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user 
##                 (first and last name) for userid = 10, sessionid = 182

query2 = "CREATE TABLE IF NOT EXISTS user_library "
query2 = query2 + """(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, 
                PRIMARY KEY ((userId, sessionId), itemInSession))"""

queries.append(query2)
    
## Query 3: Give me every user name (first and last) in my music app history who listened to the song '
## 'All Hands Against His Own'

query3 = "CREATE TABLE IF NOT EXISTS song_library "
query3 = query3 + """(song text, userId int, firstName text, lastName text, 
                PRIMARY KEY (song, userId))"""

queries.append(query3)

for query in queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)    
        

In [14]:
## checking the tables name created
a = session.execute("SELECT * FROM system_schema.tables WHERE keyspace_name = 'udacity';")
    
for row in a:
    print (row[1])
    

session_library
song_library
user_library


### Insert records into tables


In [15]:
# Insert records on a table 

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variable
        query1 = "INSERT INTO session_library (sessionId, itemInSession, artist, song, length)"
        query2 = "INSERT INTO user_library (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query3 = "Insert INTO song_library (song, userId, firstName, lastName)"
    
    #query = query + "<ASSIGN VALUES HERE>"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))
        

#### Doing a SELECT to verify that the data have been inserted into each table

In [16]:
## Transform the select query result into dataframe

pd.options.display.max_colwidth = 100

def get_dataframe(rows, selection, list_columns):
    dict_result = {}
    i = 0
  
    for row in rows:
        dict_result[i] = [content for content in row]
        i += 1
    
    result = pd.DataFrame.from_dict(dict_result, orient='index', columns=list_columns)
        
    with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
            print ('\n ------------------ Query', selection, ' Result')
            print(result)
            
    return result

In [17]:
# Query 1: 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4. 
query = "select artist, song from session_library WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
##get the dataframe from result    
query1 = get_dataframe(rows, 1, ['Artist', 'Song'])




 ------------------ Query 1  Result
      Artist                             Song
0  Faithless  Music Matters (Mark Knight Dub)


In [18]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "select artist, song, firstName, lastName from user_library WHERE userId = 10 and sessionId = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
##get the dataframe from result    
query2 = get_dataframe(rows, 2, ['Artist', 'Song', 'First_Name', 'Last_ Name'])



 ------------------ Query 2  Result
              Artist                                                  Song  \
0   Down To The Bone                                    Keep On Keepin' On   
1       Three Drives                                           Greece 2000   
2  Sebastien Tellier                                             Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio Edit)   

  First_Name Last_ Name  
0     Sylvie       Cruz  
1     Sylvie       Cruz  
2     Sylvie       Cruz  
3     Sylvie       Cruz  


In [19]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song '
## 'All Hands Against His Own'

query = "select firstName, lastName from song_library WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
##get the dataframe from result    
query3 = get_dataframe(rows, 3, ['First_Name', 'Last_Name'])


 ------------------ Query 3  Result
   First_Name Last_Name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions
queries = []
query1 = "drop table IF EXISTS session_library"
queries.append(query1)

query2 = "drop table IF EXISTS user_library"
queries.append(query2)

query3 = 'drop table IF EXISTS song_library'
queries.append(query3)

for query in queries:
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()

### Parte III - match between query result and the select done from dataframe event_datafile_new

In [20]:
#query1
event1_list = event[(event.sessionId == 338) & (event.itemInSession == 4)][['artist', 'song']].values.tolist()
query1_list = query1.values.tolist()

event1_list == query1_list

True

In [21]:
#query2
event2_list = event[(event.userId == 10) & (event.sessionId == 182)][['artist', 'song', 'firstName', 'lastName']].values.tolist()
query2_list = query2.values.tolist()

event2_list == query2_list

True

In [28]:
#q3

event3 = event[event.song == 'All Hands Against His Own'].sort_values(by=['song', 'userId'])[['firstName', 'lastName']]
event3.drop_duplicates(inplace=True)
event3_list= event3.values.tolist()

query3_list = query3.values.tolist()
event3_list == query3_list


True